In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import FashionMNIST

In [3]:
# Set parameters
batch_size = 64
learning_rate = 0.001
num_epochs = 10

In [4]:
# Data preparation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [5]:
# Load data
dataset = FashionMNIST(root='./data', train=True, download=True, transform=transform)
train_set, val_set = random_split(dataset, [50000, 10000])  # Split data into training and validation sets
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)


In [6]:
# Build MLP model
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # flatten input tensor
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

model = MLP()

In [7]:

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [8]:
#Train the model
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Training Loss: {running_loss/len(train_loader)}")


Epoch 1, Training Loss: 0.5230648009597189
Epoch 2, Training Loss: 0.38303718058501973
Epoch 3, Training Loss: 0.3418142681136308
Epoch 4, Training Loss: 0.31784385708553714
Epoch 5, Training Loss: 0.2968655277300826
Epoch 6, Training Loss: 0.27973311141018975
Epoch 7, Training Loss: 0.2636669228124954
Epoch 8, Training Loss: 0.24604773046949024
Epoch 9, Training Loss: 0.2355955044174438
Epoch 10, Training Loss: 0.22317340066346825


In [9]:
# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on validation set: {100 * correct / total}%")

Accuracy on validation set: 88.75%
